In [13]:
import csv
import pandas as pd
from getpass import getpass
from time import sleep 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome

In [ ]:
driver = Chrome(ChromeDriverManager().install())
sleep(2)
driver.get("https://twitter.com/login")

In [15]:
sleep(3)
username = driver.find_element(By.XPATH,"//input[@name='text']")
username.send_keys('rcnexrt')

sleep(1)
next_btn = driver.find_element(By.XPATH, "//span[contains(text(),'Next')]")
next_btn.click()

In [16]:
pw = getpass()

········


In [17]:
sleep(3)
password = driver.find_element(By.XPATH,"//input[@name='password']")
password.send_keys(pw)

sleep(1)
log_in = driver.find_element(By.XPATH, "//span[contains(text(),'Log in')]")
log_in.click()

In [19]:
key_word = input("Keyword apa yang ingin dicari: ")
sleep(3)
profile = driver.find_element(By.XPATH, "//input[@placeholder='Search Twitter']")
profile.send_keys(key_word)
profile.send_keys(Keys.RETURN)

# sleep(1)
# target_acc = driver.find_element(By.XPATH, "//span[contains(text(),'@abinwong')]")
# target_acc.click()

Keyword apa yang ingin dicari: apbn


In [21]:
def scraper(n_unique):
    timestamps = [] 
    names = []
    usernames = []
    tweets = []
    replys = []
    retweets = []
    likes = []

    last_position = driver.execute_script("return window.pageYOffset;")

    while True:
        articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
        for article in articles:

            usertag = driver.find_element(By.XPATH, ".//div[@data-testid='User-Names']").text
            name, username = usertag.split("\n")[:2]
            names.append(name)
            usernames.append(username)

            timestamp = driver.find_element(By.XPATH, ".//time").get_attribute("datetime")
            timestamps.append(timestamp)

            tweet = driver.find_element(By.XPATH, ".//div[@data-testid='tweetText']").text
            tweets.append(tweet)

            reply = driver.find_element(By.XPATH, ".//div[@data-testid='reply']").text
            replys.append(reply)

            like = driver.find_element(By.XPATH, ".//div[@data-testid='like']").text
            likes.append(like)

            retweet = driver.find_element(By.XPATH, ".//div[@data-testid='retweet']").text
            retweets.append(retweet)

        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
        tweets2 = list(set(tweets))
        print(len(set(tweets2)))
        sleep(1)
        
        if len(tweets2) > n_unique:
            break
    
    df = pd.DataFrame(zip(timestamps,tweets,replys,retweets,likes,names,usernames), 
                    columns=["timestamp","tweet","reply","retweet","like","name","username"])
    df = df.drop_duplicates(subset=["tweet"])
    return df

def cleaner(df):
    df = df.drop_duplicates(subset=["tweet"])
    df["date"] = pd.to_datetime(df["timestamp"]).dt.strftime("%d-%b-%Y")
    return df

In [12]:
df = scraper(100)
dfa = cleaner(df)
dfa.to_csv(f"scrape_{key_word}.csv")